In [1]:
save_dir = '/Users/yeonghyeon/Documents/Study/TAVE Research/데이터 분석 프로젝트팀/2. Tourism/data/'

In [2]:
import time, os
from selenium import webdriver
import requests
from bs4 import BeautifulSoup

from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.alert import Alert
import time

import tqdm
import pandas as pd

import re

In [3]:
# 드라이버 지정
chrome_path = '/opt/homebrew/bin/chromedriver'
options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성
options.add_argument("enable-automation")
#options.add_argument('headless') # headless 모드 설정 (창 안 띄움)
options.add_argument("window-size=1920x1080") # 화면크기
options.add_argument("--no-sandbox")
options.add_argument("disable-gpu") 
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--dns-prefetch-disable")
options.add_argument('--disable-site-isolation-trials')


In [6]:
#Step 1. 크롬 웹브라우저 실행
driver = webdriver.Chrome(chrome_path, options=options)

In [ ]:
naver_df = pd.DataFrame(columns=['날짜', '제목', '내용', 'url'])
query = "제주 여행"

driver.get('http://blog.naver.com')
soup = BeautifulSoup('http://blog.naver.com')

#Step 2. 네이버 검색창에 "검색어" 검색
element = driver.find_element_by_xpath("""//*[@id="header"]/div[1]/div/div[2]/form/fieldset/div/input""")
element.send_keys(query, Keys.ENTER)  # query_txt는 위에서 입력한 '이재용'
#element.send_keys(Keys.ENTER)
time.sleep(1)

#Step 3. 기간 선택
#element_date = driver.find_element_by_link_text("기간 전체")
element_date = driver.find_element_by_css_selector("#content > section > div.category_search > div.search_information > div > div > a")
element_date.send_keys(Keys.ENTER)

#Step 4. 날짜 입력
# 시작 날짜 입력하기
element_start = driver.find_element_by_css_selector("#search_start_date")
element_start.click()
time.sleep(1)
element_start.send_keys('2021-06-01')
# 빈 곳 클릭
driver.find_element_by_css_selector("#content > section > div.category_search > div.search_information > div > div > div > a:nth-child(4)").click()

# 종료 날짜 입력하기
element_end = driver.find_element_by_css_selector("#search_end_date")
element_end.click()
time.sleep(1)
element_end.send_keys('2021-06-30')
# 빈 곳 클릭
driver.find_element_by_css_selector("#content > section > div.category_search > div.search_information > div > div > div > a:nth-child(4)").click()


# Step 5. 날짜 입력 후 "적용" 버튼 클릭
driver.find_element_by_xpath("""//*[@id="periodSearch"]""").click()
time.sleep(0.5)

# 월별 검색 건수의 20% 크롤링
total_page = 0
btn_cnt = 1

cnt_per_month = driver.find_elements_by_class_name("search_number")[0].text
total_cnt = int(int(cnt_per_month.split('건')[0].split(',')[0] + cnt_per_month.split('건')[0].split(',')[1]) * 0.2)

while total_page != total_cnt:
    # 한 페이지 안의 총 기사 개수
    t_len = len(driver.find_elements_by_class_name("list_search_post"))

    for i in range(1, t_len+1):
        a = driver.find_element_by_css_selector("#content > section > div.area_list_search > div:nth-child({}) > div > div.info_post > div.desc > a.desc_inner".format(i))

        # url 수집
        url = a.get_attribute('href')

        # 제목 수집
        title = driver.find_element_by_css_selector("#content > section > div.area_list_search > div:nth-child({}) > div > div.info_post > div.desc > a.desc_inner > strong > span".format(i)).text
        
        naver_df.loc[total_page] = ['nan', title, 'nan', url]

        total_page += 1
        print(total_page)
    
    btn_cnt += 1

    # 다음 버튼 누르기
    if btn_cnt % 10 == 1:
        if btn_cnt > 11:
            btn = driver.find_element_by_css_selector("#content > section > div.pagination > a.button_next")
            time.sleep(0.6)
            btn.send_keys(Keys.ENTER)
        else:
            btn = driver.find_element_by_css_selector("#content > section > div.pagination > a")
            btn.send_keys(Keys.ENTER)
        time.sleep(1)
    else:
        if btn_cnt > 11:
            if (btn_cnt % 10) == 0:
                btn = driver.find_element_by_css_selector("#content > section > div.pagination > span:nth-child(11) > a")
            else:
                btn = driver.find_element_by_css_selector("#content > section > div.pagination > span:nth-child({}) > a".format((btn_cnt%10)+1))
        else:
            btn = driver.find_element_by_css_selector("#content > section > div.pagination > span:nth-child({}) > a".format(btn_cnt))
        time.sleep(0.6)
        btn.send_keys(Keys.ENTER)
        time.sleep(1)
    

In [8]:
naver_df.drop_duplicates(inplace=True)
naver_df = naver_df.reset_index(drop=True)
naver_df

,날짜,제목,내용,url
0,nan,제주택시투어로 돌아본 제주여행코스 ft.골든택시,nan,https://blog.naver.com/nau2001/222392206200
1,nan,제주도 렌트카 제주 여행 꿀팁 추천,nan,https://blog.naver.com/tcacyc/222405921669
2,nan,『제주 여행』 오설록 기념품도 사고 사진도 찍고,nan,https://blog.naver.com/irelsuga/222393929984
3,nan,돌고래 요트 투어 제주여행에서 즐긴 M1971,nan,https://blog.naver.com/k4minlove/222414762982
4,nan,2박3일 제주여행 발이되어준 제주도렌트카 후기,nan,https://blog.naver.com/hyr67/222411712257
...,...,...,...,...
3995,nan,"제주여행, 하마다게스트하우스 조식 룸서비스 210602, 와이케이언니의 서른여덟번째...",nan,https://blog.naver.com/hamadajeju/222381090960
3996,nan,광주-제주 3박 4일 일정. 제주여행,nan,https://blog.naver.com/cotl0208/222413539997
3997,nan,2021 오월의 제주｜혼자 제주 여행 (Day 3. 신창풍차해안도로 자전거 / 카페...,nan,https://blog.naver.com/baedudals/222413079286
3998,nan,제주 여행 04,nan,https://blog.naver.com/s__e/222403355309


In [ ]:
# 2차 크롤링 - 블로그 본문, 포스팅 날짜
import time
start = time.time()

for i in range(len(naver_df3)):
    content_list = ""
    driver.get(naver_df3.iloc[i]['url'])
    driver.switch_to.frame('mainFrame')
        
    try:
        time.sleep(1)
        #overlays = ".se-component.se-text.se-l-default" # 내용 크롤링
        contents = driver.find_elements_by_css_selector(".se-component.se-text.se-l-default")
        for content in contents:
            content_list = content_list + content.text
            
    except():
        time.sleep(1)
        contents = driver.find_elements_by_css_selector("#postViewArea")
        for content in contents:
            for c in content:
                content_list = content_list + c.text

    # date 수집
    try:
        date = driver.find_element_by_css_selector(".se_publishDate.pcol2").text
    except:
        date = driver.find_element_by_css_selector(".date.fil5.pcol2._postAddDate").text
    
    naver_df3.iloc[i][['날짜', '내용']] = [date, content_list]
    
    print(i)
    
print(time.time()-start)

In [25]:
import pandas as pd

naver_final = pd.read_csv(save_dir+'naver_keword.csv')
naver_final = pd.concat([naver_final, naver_df])
naver_final.reset_index(drop=True, inplace=True)
naver_final.to_csv(save_dir + 'naver_keword.csv', index=False)